## <span style="display: block; text-align: center; font-family: verdana; color: purple;">* Feature Extraction (part 01) for Recipe Recommendar System *


                                                                                                                           
                                                                                                                        By,
                                                                                                                         - Abhishek Kurri
                                                                                                                         - Bidisha Bose Mukherjee
                                                                                                                         - Sejal Sahu
    
 </span>

### <span style="font-family: verdana; color: dimgrey;">**INITIATING SPARK SESSION...**</span>

In [2]:
from pyspark.sql import SparkSession

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("recipe_recommendation").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.sql import functions as F

# Import for typecasting columns

from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType,StringType

from pyspark.sql.types import ArrayType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ### <span style="font-family: verdana; color: royalblue;">TASK 01: READ THE DATA (RAW_RECIPES_CLEANED)</span>

In [6]:
# File location can vary, here we are using S3 Bucket path

recipe_df = spark.read.options(
    inferSchema='True',
    header='True').csv(
    "s3://aws-emr-resources-294075675661-us-east-1/data/RAW_recipes_cleaned.csv"
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# show data 

recipe_df.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|                name|    id|minutes|contributor_id|          submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|
+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|arriba   baked wi...|137739|     55|         47892|2005-09-16 00:00:00|['60-minutes-or-l...|[51.5, 0.0, 13.0,...|     11|['make a choice a...|autumn is my favo...|['winter squash',...|            7|
|a bit different  ...| 31490|     30|         26278|2002-06-17 00:00:00|['30-minutes-or-l...|[173.4, 18.0, 0.0...|      9|['preheat oven to...|this recipe calls...|['prepared pizza ...|            6|


In [8]:
#  checking the df columns defination

recipe_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)

### <span style="font-family: verdana; color: dimgrey;">TEST CASES FOR TASK 01</span>

In [9]:
# Code check cell

assert recipe_df.count() == 231637, "There is a mistake in reading the data."
assert len(recipe_df.columns) == 12, "There is a mistake in reading the data."
assert recipe_df.schema["minutes"].dataType == IntegerType(), "The data types have not been read correctly."
assert recipe_df.schema["tags"].dataType == StringType(), "The data types have not been read correctly."
assert recipe_df.schema["n_ingredients"].dataType == IntegerType(), "The data types have not been read correctly."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 <span style="font-family: verdana; color: royalblue;">All checks are passed.</span>

### <span style="font-family: verdana; color: dimgrey;">EXTRACT ```NUTRITION``` VALUES</span>

In [10]:
# List of nutrition columns

nutrition_column_names = ['calories',
                          'total_fat_PDV',
                          'sugar_PDV',
                          'sodium_PDV',
                          'protein_PDV',
                          'saturated_fat_PDV',
                          'carbohydrates_PDV']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ### <span style="font-family: verdana; color: royalblue;">Task 02: Extract individual features from the nutrition column.</span>

<span style="font-family: verdana; color: dimgrey;">
    
   - As read by the spark compiler, the nutrition column is a string column when it should be an array of float values.
   - Each row in the nutrition column contains seven values. Each value represents nutrition information.   
    
**Your task is to separate the array into seven individual columns.**
   - Write a code that takes in the nutrition column from ```raw_recipes_df``` dataframe, 
   - and extracts individual values into seven different columns named calories, total fat (PDV), 
   - sugar (PDV), sodium (PDV), protein (PDV), saturated fat (PDV), and carbohydrates (PDV).

</span>

#### <span style="font-family: verdana; color: green;">Solution:</span>

In [11]:
recipe_df.select('nutrition').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           nutrition|
+--------------------+
|[51.5, 0.0, 13.0,...|
|[173.4, 18.0, 0.0...|
|[269.8, 22.0, 32....|
|[368.1, 17.0, 10....|
|[352.9, 1.0, 337....|
+--------------------+
only showing top 5 rows

#### <span style="font-family: verdana; color: dimgrey;"> Task 2.1
    - Use string operations to remove the square brackets from the nutrition column. 
</span>

In [12]:
# importing regular expression package

from pyspark.sql.functions import regexp_replace

recipe_df = recipe_df.withColumn("nutrition",regexp_replace("nutrition","\\[|\\]",""))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# output of above cell, as of now we are able to replace the [] with ""

recipe_df.select('nutrition').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           nutrition|
+--------------------+
|51.5, 0.0, 13.0, ...|
|173.4, 18.0, 0.0,...|
|269.8, 22.0, 32.0...|
|368.1, 17.0, 10.0...|
|352.9, 1.0, 337.0...|
+--------------------+
only showing top 5 rows

#### <span style="font-family: verdana; color: royalblue;">**Task 2.1 is completed.**</span>

#### <span style="font-family: verdana; color: dimgrey;"> Task 2.2: 
    - Split the nutrition column into seven individual columns and cast the new columns to float values. 
    - First split the column on using the comma delimiter. 
    - Then you can use a for loop to iterate over the column names declared in the variable nutrition_column_names, 
    - inside each iteration write a code to extract the value at a specific index of the nutrition array  
    
</span>

In [14]:
# creating variable to split nutrition columns values

import pyspark

nutrition_cols_split = pyspark.sql.functions.split(recipe_df['nutrition'],',') 

# loop to extract individual values from the nutrition column

for col_index, col_name in enumerate(nutrition_column_names):    
    recipe_df = (recipe_df.withColumn(col_name,nutrition_cols_split.getItem(col_index).cast("float")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
recipe_df.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+
|                name|    id|minutes|contributor_id|          submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|
+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+
|arriba   baked wi...|137739|     55|         47892|2005-09-16 00:00:00|['60-minutes-or-l...|51.5, 0.0, 13.0, ...|     11|['

In [16]:
recipe_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_PDV: float (nullable = true)
 |-- sugar_PDV: float (nullable = true)
 |-- sodium_PDV: float (nullable = true)
 |-- protein_PDV: float (nullable = true)
 |-- saturated_fat_PDV: float (nullable = true)
 |-- carbohydrates_PDV: float (nullable = true)

<span style="font-family: verdana; color: dimgrey;"> **Test cases for task 02** </span>

In [17]:
# Code check cell

assert recipe_df.schema["carbohydrates_PDV"].dataType == FloatType(), "Recheck your typecasting"
assert recipe_df.collect()[123432][14] == 62.0, "The columns have not been split correctly."
assert recipe_df.collect()[10000][12] == 60.400001525878906, "The columns have not been split correctly."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<span style="font-family: verdana; color: royalblue;">All checks are passed, Task 2.2 is completed.</soan>

### <span style="font-family: verdana; color: royalblue;">Task 03: Standardize the nutrition values </span>

<span style="font-family: verdana; color: dimgrey;">
    
The current values for nutrition columns are not on the same scale. 
Your task is to standardize the nutrition columns using calories as the base of standardization. 

Convert the nutrition from absolute values to per 100 calorie values. 

</span>

<span style="font-family: verdana; color: dimgrey;">
    
We will use the  ```sugar (PDV)``` column to demonstrate the calculations for standardization.  

</span>

<span style="font-family: verdana; color: dimgrey;">
    
**Sample Calculation**

Before transformation: ```sugar (PDV)``` for recipe id 137739 = 13.0

Calories in the recipe recipe id 137739                       = 51.5

Calculation:  
sugar_per_100_cal = 13.0 * 100 / 51.5 

After transformation ```sugar_per_100_cal``` = 25.24
    
</span>

#### <span style="font-family: verdana; color: green;">Solution:</span>

In [18]:
# Task 03 Cell 1 out of 1

for nutri_col in nutrition_column_names:
    if nutri_col != "calories":
        # following code will name the new columns 
        nutrition_per_100_cal_col = (nutri_col.replace('_PDV','') + '_per_100_cal')
        
        recipe_df = recipe_df.withColumn(nutrition_per_100_cal_col,recipe_df[nutri_col]*100/recipe_df["calories"])
        
        # You might end up adding nulls to the data because of our intended transformation. 
        # Perform a fill na operation to fill all the nulls with 0s. 
        # You must limit the scope of the fill na to the current column only. 
                
        recipe_df = recipe_df.fillna(value=0,subset=[nutrition_per_100_cal_col]) 
        # pyspark code to fill nulls with 0 in only the current nutrition_per_100_cal_col 
        

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
recipe_df.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+---------------------+-----------------+------------------+-------------------+-------------------------+-------------------------+
|                name|    id|minutes|contributor_id|          submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|total_fat_per_100_cal|sugar_per_100_cal|sodium_per_100_cal|protein_per_100_cal|saturated_fat_per_100_cal|carbohydrates_per_100_cal|
+--------------------+------+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+-----------

In [20]:
recipe_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_PDV: float (nullable = true)
 |-- sugar_PDV: float (nullable = true)
 |-- sodium_PDV: float (nullable = true)
 |-- protein_PDV: float (nullable = true)
 |-- saturated_fat_PDV: float (nullable = true)
 |-- carbohydrates_PDV: float (nullable = true)
 |-- total_fat_per_100_cal: double (nullable = false)
 |-- sugar_per_100_cal: double (nullable = false)
 |-- sodium_per_100_cal: double (nullable = false)
 |-- protein_per_100_cal: double (nullable = false

<span style="font-family: verdana; color: dimgrey;"> **Test cases for Task 03** </span>

In [21]:
# total fat check for id 28881
assert recipe_df.filter("id == 28881").select('total_fat_per_100_cal').first()[0] == 0, "total_fat_per_100_cal for recipe 28881 should be 0"

# total fat check for id 112140
assert round(recipe_df.filter("id == 112140").select('total_fat_per_100_cal').first()[0]) == 8, "total_fat_per_100_cal for recipe 112140 should be 8"

# checking for nulls
for c in ['total_fat_per_100_cal','sugar_per_100_cal','sodium_per_100_cal','protein_per_100_cal','saturated_fat_per_100_cal','carbohydrates_per_100_cal']:
    assert recipe_df.select(F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c)).collect()[0][0] == 0, "There are Nulls in the data"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<span style="font-family: verdana; color: royalblue;">All checks are passed, TASK 03 is completed.</span>

### <span style="font-family: verdana; color: royalblue;">Task 04: Convert the tags column from a string to an array of strings </span>

<span style="font-family: verdana; color: dimgrey;">
    
- Currently, the tags column is a string column but holds an array of strings. 

- Your task is to convert the tags columns from a string to an array of strings. 
    
</span>

<span style="font-family: verdana; color: dimgrey;">

- Remove ```[``` ```]``` ```'``` punctuation marks from the tags column. 
Split the tags column based on the comma delimiter. 
    

</span>

#### <span style="font-family: verdana; color: green;">Solution</span>

In [22]:
# pyspark function to remove symbols like '[' ']' "'" from the strings in the tags column.
# pyspark function to split the column using the comma delimiter.

from pyspark.sql.functions import split, regexp_replace
from pyspark.sql.types import ArrayType, StringType

# Remove [ ] ' punctuation marks from the tags column
recipe_df = recipe_df.withColumn('tags', regexp_replace("tags", "[\\[\\]\\']", ""))

# Split the tags column into an array of strings
recipe_df = recipe_df.withColumn("tags", split(recipe_df["tags"], ", ").cast(ArrayType(StringType())))

# Verify the type of the tags column
# assert recipe_df.schema["tags"].dataType == ArrayType(StringType(), True), "You have not split the string into an array."


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
recipe_df.select("tags").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|                tags|
+--------------------+
|[60-minutes-or-le...|
|[30-minutes-or-le...|
|[time-to-make, co...|
|[60-minutes-or-le...|
|[weeknight, time-...|
|[15-minutes-or-le...|
|[15-minutes-or-le...|
|[weeknight, time-...|
|[weeknight, time-...|
|[weeknight, time-...|
+--------------------+
only showing top 10 rows

In [24]:
recipe_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: timestamp (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_PDV: float (nullable = true)
 |-- sugar_PDV: float (nullable = true)
 |-- sodium_PDV: float (nullable = true)
 |-- protein_PDV: float (nullable = true)
 |-- saturated_fat_PDV: float (nullable = true)
 |-- carbohydrates_PDV: float (nullable = true)
 |-- total_fat_per_100_cal: double (nullable = false)
 |-- sugar_per_100_cal: double (nullable = false)
 |-- sodium_per_100_cal: double (nullable = false)
 |-

<span style="font-family: verdana; color: dimgrey;"> **Test cases for Task 04** </span>

In [25]:

assert recipe_df.schema["tags"].dataType == ArrayType(StringType(), True), "You have not split the string into an array."
assert recipe_df.collect()[2][5] == ['time-to-make','course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less'], "Recheck your string cleaning and splitting operations."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 <span style="font-family: verdana; color: royalblue;">All checks are passed, TASK 04 is completed.</span>

### <span style="font-family: verdana; color: dimgrey;">Join Recipe Data to Review Data</span>

In [26]:
# Reading the second data set. 

ratings_df = (spark.read.csv("s3://aws-emr-resources-294075675661-us-east-1/data/RAW_interactions_cleaned.csv", 
                                 header=True, 
                                 inferSchema= True)
                  .withColumn("review_date",  F.col("date"))
                  .drop(F.col("date"))
                  )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
ratings_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: timestamp (nullable = true)

In [28]:
# Code check cell

assert ratings_df.count() == 1132367, "There is a mistake in reading the data."
assert len(ratings_df.columns) == 5, "There is a mistake in reading the data."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
ratings_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------+--------------------+-------------------+
|user_id|recipe_id|rating|              review|        review_date|
+-------+---------+------+--------------------+-------------------+
|  38094|    40893|     4|Great with a sala...|2003-02-17 00:00:00|
|1293707|    40893|     5|So simple  so del...|2011-12-21 00:00:00|
|   8937|    44394|     4|This worked very ...|2002-12-01 00:00:00|
| 126440|    85009|     5|I made the Mexica...|2010-02-27 00:00:00|
|  57222|    85009|     5|Made the cheddar ...|2011-10-01 00:00:00|
+-------+---------+------+--------------------+-------------------+
only showing top 5 rows

### <span style="font-family: verdana; color: royalblue;">Task 05: Read the second data file </span>

<span style="font-family: verdana; color: dimgrey;">

- Along with raw recipes data, we also have raw ratings data. 

- The code to read the data is already written above. Your task is to join the raw ratings and raw recipes data. 

- The resulting dataframe must have the same number of rows as in the raw ratings data. 
    
- Join both the dataframes using the recipie IDs.
    
</span>

<span style="font-family: verdana; color: dimgrey;">

**Calculation explanation** 

There are 25 columns in the ```raw_recipes_df``` and five in the ```raw_ratings_df```. So total columns in the combined dataframe 25 + 5 = 30

The number of rows in the combined dataframe must be the same as the rows in the ```raw_ratings_df```. So total rows in combined dataframe 1132367
    
</span>

#### <span style="font-family: verdana; color: green;">Solution</span>

In [30]:
# performing inner join between 2 dataframes

join_df = ratings_df.join(recipe_df,ratings_df.recipe_id == recipe_df.id,"inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
join_df.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------+--------------------+-------------------+--------------------+---+-------+--------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+---------------------+-------------------+------------------+-------------------+-------------------------+-------------------------+
|user_id|recipe_id|rating|              review|        review_date|                name| id|minutes|contributor_id|          submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|total_fat_per_100_cal|  sugar_per_100_cal|sodium_per_100_cal|protein_per_100_cal|saturated_fat_per_100_cal|carbohydrates_per_100_cal|
+-------+---------+-----

<span style="font-family: verdana; color: dimgrey;"> **Test cases for Task 05** </span>

In [32]:
assert (join_df.count() ,len(join_df.columns)) == (1132367, 30), "The type of join is incorrect"

list1 = ratings_df.select('recipe_id').collect()
list2 = recipe_df.select('id').collect()
exclusive_set = set(list1)-set(list2)

assert len(exclusive_set) == 0, "There is a mistake in reading one of the two data files."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 <span style="font-family: verdana; color: royalblue;">All checks are passed, TASK 05 is completed.</span>

### <span style="font-family: verdana; color: royalblue;">Task 06:  Create time-based features</span>

<span style="font-family: verdana; color: dimgrey;">

Currently, both the date columns, the submitted date, and the review date are in string forms. 
    
First convert the ```submitted``` and ```review_date``` to DateType()

Use review date and submission date to derive new features:
1. ```days_since_submission_on_review_date``` Number of days between the recipe submission and the current review.  
2. ```months_since_submission_on_review_date``` Number of months between the recipe submission and the current review. 
3. ```years_since_submission_on_review_date```Number of years between the recipe submission and the current review. 

</span>

<span style="font-family: verdana; color: dimgrey;">

<span style="font-family: verdana; color: green;"> **Sample Calculation** </span>

Recipe 40893 was submitted on 2002-09-21
User 38094 reviewed recipe 40893 on 2003-02-17

```days_since_submission_on_review_date``` number of calender days between 2002-09-21 and 2003-02-17 that is 149

```months_since_submission_on_review_date``` number of calender months between 2002-09-21 and 2003-02-17 that is 4.87 (calculated by a pyspark function)

```years_since_submission_on_review_date``` number of calender months divided by 12 that is 0.40
    
</span>

#### <span style="font-family: verdana; color: green;">Solution</span>

In [33]:
# Task 06 Cell 1 out of 2

join_df = (join_df
                        .withColumn('submitted', F.col("submitted").cast('date') # pyspark function to cast a column to DateType()
                                   )
                        .withColumn('review_date', F.col("review_date").cast('date')# pyspark function to cast a column to DateType()
                                   )
                                             
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# checking whether the both the fields converted to date type

join_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: date (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_PDV: float (nullable = true)
 |-- sugar_PDV: float (nullable = true)
 |-- sodium_PDV: float (nullable = true)
 |-- protein_PDV: float (nullable = true)
 |-- saturated_fat_PDV: float (nullable = true)
 |-- carboh

In [36]:
join_df = (join_df
                        .withColumn('days_since_submission_on_review_date',F.datediff("review_date","submitted")
                                     # Pyspark function to find the number of days between two dates              
                                   )
                        .withColumn('months_since_submission_on_review_date',F.months_between("review_date","submitted")
                                     # Pyspark function to find the number of months between two dates          
                                   )
                        .withColumn('years_since_submission_on_review_date',F.months_between("review_date","submitted")/12
                                     # Pyspark function to find the number of months between two dates / 12          
                                   )
                         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
join_df.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------+--------------------+-----------+--------------------+---+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+---------------------+-------------------+------------------+-------------------+-------------------------+-------------------------+------------------------------------+--------------------------------------+-------------------------------------+
|user_id|recipe_id|rating|              review|review_date|                name| id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|total_fat_per_100_cal|  sugar_per_100_cal|sodium_per_100_cal|protein_per_100_c

<span style="font-family: verdana; color: dimgrey;"> **Test cases for Task 06** </span>

In [38]:
#  validation

assert join_df.schema["days_since_submission_on_review_date"].dataType == IntegerType()

assert (join_df.filter((join_df.user_id == 428885) & (join_df.recipe_id == 335241))
                            .select('days_since_submission_on_review_date').collect()[0][0]) == 77

assert (join_df.filter((join_df.user_id == 2025676) & (join_df.recipe_id == 94265))
                            .select('months_since_submission_on_review_date').collect()[0][0]) == 153.22580645

assert (join_df.filter((join_df.user_id == 338588) & (join_df.recipe_id == 21859))
                            .select('years_since_submission_on_review_date').collect()[0][0]) == 4.564516129166667

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 <span style="font-family: verdana; color: royalblue;">All checks are passed, TASK 06 is completed.</span>

In [42]:
join_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: date (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_PDV: float (nullable = true)
 |-- sugar_PDV: float (nullable = true)
 |-- sodium_PDV: float (nullable = true)
 |-- protein_PDV: float (nullable = true)
 |-- saturated_fat_PDV: float (nullable = true)
 |-- carboh

In [43]:
assert (join_df.count() ,len(join_df.columns) ) == (1132367, 33)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<span style="font-family: verdana; color: royalblue;"> Saving the file in parquet format </span>

In [45]:
# writing the parquet file in s3 path, File location can vary...

join_df.write.mode('overwrite').parquet('s3://aws-emr-resources-294075675661-us-east-1/data/join_df_processed.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<span style="font-family: verdana; color: royalblue"> Feature Extraction (part 01) is completed. </span>